In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df=pd.read_csv('../input/predict-test-scores-of-students/test_scores.csv')

In [ ]:
df.head()

# Exploring the dataset

In [ ]:
df.describe

In [ ]:
df.value_counts()

In [ ]:
df.dtypes

In [ ]:
df.isnull().sum()

# Vertical Bar Chart: No. of students in schools.
# Horizontal Bar Chart: Marks obtained in PreTest V/S PostTest on basis of class.
# Pie Chart: Percentage of PreTest vs PostTest marks obtained on basis of School Type.
# Packed Bubbles: Percentage of PreTest vs PostTest marks obtained on basis of School Setting.

<div class='tableauPlaceholder' id='viz1629134224160' style='position: relative'><noscript><a href='#'><img alt='Dashboard 1 ' src='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;Sc&#47;Schooltestscores&#47;Dashboard1&#47;1_rss.png' style='border: none' /></a></noscript><object class='tableauViz'  style='display:none;'><param name='host_url' value='https%3A%2F%2Fpublic.tableau.com%2F' /> <param name='embed_code_version' value='3' /> <param name='site_root' value='' /><param name='name' value='Schooltestscores&#47;Dashboard1' /><param name='tabs' value='no' /><param name='toolbar' value='yes' /><param name='static_image' value='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;Sc&#47;Schooltestscores&#47;Dashboard1&#47;1.png' /> <param name='animate_transition' value='yes' /><param name='display_static_image' value='yes' /><param name='display_spinner' value='yes' /><param name='display_overlay' value='yes' /><param name='display_count' value='yes' /><param name='language' value='en-US' /></object></div>                <script type='text/javascript'>                    var divElement = document.getElementById('viz1629134224160');                    var vizElement = divElement.getElementsByTagName('object')[0];                    if ( divElement.offsetWidth > 800 ) { vizElement.style.width='1366px';vizElement.style.height='1051px';} else if ( divElement.offsetWidth > 500 ) { vizElement.style.width='1366px';vizElement.style.height='1051px';} else { vizElement.style.width='100%';vizElement.style.height='1927px';}                     var scriptElement = document.createElement('script');                    scriptElement.src = 'https://public.tableau.com/javascripts/api/viz_v1.js';                    vizElement.parentNode.insertBefore(scriptElement, vizElement);                </script>

# We have no missing data.
## Changing the objects to numerical data types

In [ ]:
df.columns

In [ ]:
objects=pd.get_dummies(df[['school_setting', 'school_type', 'classroom', 'teaching_method', 'student_id', 'gender', 'lunch']])

In [ ]:
objects.head()

In [ ]:
df=pd.concat([df[['n_student','pretest', 'posttest']],objects],axis=1)

In [ ]:
df.head()

In [ ]:
df.shape

# Building the model

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler

In [ ]:
x=df.drop('posttest',axis=1)
y=df['posttest']

## Scaling the Dataset

In [ ]:
scaler=StandardScaler()
x=scaler.fit_transform(x)
xtr,xte,ytr,yte=train_test_split(x,y,random_state=42,test_size=0.3)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
reg=RandomForestRegressor(n_jobs=-1,verbose=2)

In [ ]:
param_grid={'n_estimators':[10,30,60,100,150,170,200],'criterion':['mse','mae'],'max_depth':[1,2,3,4,5,6,7,8],'min_samples_split':[2,3,4],'max_features':['auto','sqrt','log2',2,3,4,5]}

# HyperTuning parameters

In [ ]:
from sklearn.model_selection import GridSearchCV
gs=GridSearchCV(reg,param_grid=param_grid,cv=3,n_jobs=-1,verbose=1)
gs.fit(xtr,ytr)

In [ ]:
gs.best_params_

In [ ]:
gs.best_score_

In [ ]:
predicted_y=gs.predict(xte)

Testing the accuracy of model

In [ ]:
from sklearn.metrics import mean_squared_error,r2_score

In [ ]:
mean_squared_error(yte,predicted_y)

## R2 score

In [ ]:
r2_score(yte,predicted_y)

95% Accuracy Achieved!